In [24]:
import pandas as pd
import json

### Vaccination Rates

In [34]:
with open("data/2021-06-20.json") as f:
    d = json.load(f)
df_vax = pd.DataFrame(d["vaccination_county_condensed_data"])

In [38]:
df_vax.head()

,Date,FIPS,StateName,StateAbbr,County,Series_Complete_18Plus,Series_Complete_18PlusPop_Pct,Series_Complete_65Plus,Series_Complete_65PlusPop_Pct,Series_Complete_Yes,...,Administered_Dose1_Recip_65PlusPop_Pct,Administered_Dose1_PopPct_SVI,Administered_Dose1_12PlusPop_Pct_SVI,Administered_Dose1_18PlusPop_Pct_SVI,Administered_Dose1_65PlusPop_Pct_SVI,Series_Complete_Pop_Pct_SVI,Series_Complete_12PlusPop_Pct_SVI,Series_Complete_18PlusPop_Pct_SVI,Series_Complete_65PlusPop_Pct_SVI,SVI_CTGY
0,2021-06-19,01001,Alabama,AL,Autauga County,13216.0,30.8,4799.0,53.8,13470.0,...,67.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-06-19,01003,Alabama,AL,Baldwin County,63308.0,36.0,29994.0,64.0,64146.0,...,79.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-06-19,01005,Alabama,AL,Barbour County,5647.0,28.8,2791.0,57.4,5673.0,...,67.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-06-19,01007,Alabama,AL,Bibb County,4192.0,23.5,1841.0,49.3,4232.0,...,58.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-06-19,01009,Alabama,AL,Blount County,10333.0,23.2,4620.0,42.7,10438.0,...,51.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
keep_cols = df_vax.columns[[1, 3, 5, 6, 7, 8, 9, 10, 12]]
df = df_vax[keep_cols]

### Education Levels

In [55]:
edu = pd.read_excel("data/Education.xls", usecols=[0, 1, 2, 46], skiprows=4,
                   dtype={'FIPS Code':str})
edu.rename(columns={"FIPS Code":"FIPS", 
                    "Percent of adults with a bachelor's degree or higher, 2015-19":"pct_college"},
                    inplace=True)
edu = edu[["FIPS", "pct_college"]]
edu.head()

,FIPS,pct_college
0,00000,32.145542
1,01000,25.468332
2,01001,26.571573
3,01003,31.862459
4,01005,11.578713


In [92]:
# results in losing data on Guam and "unknown" counties
df1 = df.merge(edu, on="FIPS", how="inner")

In [93]:
df1.shape

(3219, 10)

In [94]:
df1.head()

,FIPS,StateAbbr,Series_Complete_18Plus,Series_Complete_18PlusPop_Pct,Series_Complete_65Plus,Series_Complete_65PlusPop_Pct,Series_Complete_Yes,Series_Complete_Pop_Pct,Census2019_12PlusPop,pct_college
0,01001,AL,13216.0,30.8,4799.0,53.8,13470.0,24.1,47574.0,26.571573
1,01003,AL,63308.0,36.0,29994.0,64.0,64146.0,28.7,192649.0,31.862459
2,01005,AL,5647.0,28.8,2791.0,57.4,5673.0,23.0,21404.0,11.578713
3,01007,AL,4192.0,23.5,1841.0,49.3,4232.0,18.9,19480.0,10.378526
4,01009,AL,10333.0,23.2,4620.0,42.7,10438.0,18.1,49234.0,13.093413


### Demographic Data

In [83]:
def clean_fips(x):
    if len(x) < 5:
        return "0" + x
    else:
        return x

In [88]:
census = pd.read_csv("data/acs2015_county_data.csv", dtype={'CensusId':str})
census.rename({"CensusId":"FIPS"}, axis=1, inplace=True)
census["FIPS"] = census["FIPS"].apply(clean_fips)

In [89]:
census.head()

,FIPS,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,01001,Alabama,Autauga,55221,26745,28476,2.6,75.8,18.5,0.4,...,0.5,1.3,1.8,26.5,23986,73.6,20.9,5.5,0.0,7.6
1,01003,Alabama,Baldwin,195121,95314,99807,4.5,83.1,9.5,0.6,...,1.0,1.4,3.9,26.4,85953,81.5,12.3,5.8,0.4,7.5
2,01005,Alabama,Barbour,26932,14497,12435,4.6,46.2,46.7,0.2,...,1.8,1.5,1.6,24.1,8597,71.8,20.8,7.3,0.1,17.6
3,01007,Alabama,Bibb,22604,12073,10531,2.2,74.5,21.4,0.4,...,0.6,1.5,0.7,28.8,8294,76.8,16.1,6.7,0.4,8.3
4,01009,Alabama,Blount,57710,28512,29198,8.6,87.9,1.5,0.3,...,0.9,0.4,2.3,34.9,22189,82.0,13.5,4.2,0.4,7.7


In [90]:
df1.head()

,FIPS,StateName,StateAbbr,Series_Complete_18Plus,Series_Complete_18PlusPop_Pct,Series_Complete_65Plus,Series_Complete_65PlusPop_Pct,Series_Complete_Yes,Series_Complete_Pop_Pct,Census2019_12PlusPop,pct_college
0,01001,Alabama,AL,13216.0,30.8,4799.0,53.8,13470.0,24.1,47574.0,26.571573
1,01003,Alabama,AL,63308.0,36.0,29994.0,64.0,64146.0,28.7,192649.0,31.862459
2,01005,Alabama,AL,5647.0,28.8,2791.0,57.4,5673.0,23.0,21404.0,11.578713
3,01007,Alabama,AL,4192.0,23.5,1841.0,49.3,4232.0,18.9,19480.0,10.378526
4,01009,Alabama,AL,10333.0,23.2,4620.0,42.7,10438.0,18.1,49234.0,13.093413


In [126]:
df2 = df1.merge(census, on="FIPS", how="inner")
df2.drop("State", axis=1, inplace=True)
df2.rename({"StateAbbr":"state", "County":"county"}, inplace=True, axis=1)
df2["state"] = df2["state"].apply(lambda x: x[:2])

In [127]:
df2.columns

Index(['FIPS', 'state', 'Series_Complete_18Plus',
       'Series_Complete_18PlusPop_Pct', 'Series_Complete_65Plus',
       'Series_Complete_65PlusPop_Pct', 'Series_Complete_Yes',
       'Series_Complete_Pop_Pct', 'Census2019_12PlusPop', 'pct_college',
       'county', 'TotalPop', 'Men', 'Women', 'Hispanic', 'White', 'Black',
       'Native', 'Asian', 'Pacific', 'Citizen', 'Income', 'IncomeErr',
       'IncomePerCap', 'IncomePerCapErr', 'Poverty', 'ChildPoverty',
       'Professional', 'Service', 'Office', 'Construction', 'Production',
       'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp', 'WorkAtHome',
       'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork', 'SelfEmployed',
       'FamilyWork', 'Unemployment'],
      dtype='object')

### Election Data

In [110]:
votes = pd.read_csv("data/county_statistics.csv", index_col=0)
keep_cols = votes.columns[[0, 1, 7, 8, 9, 10, 11]]
votes = votes[keep_cols]
votes.head()

,county,state,percentage20_Donald_Trump,percentage20_Joe_Biden,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden
0,Abbeville,SC,0.661,0.330,12433.0,8215.0,4101.0
1,Acadia,LA,0.795,0.191,28425.0,22596.0,5443.0
2,Accomack,VA,0.542,0.447,16938.0,9172.0,7578.0
3,Ada,ID,0.504,0.465,259389.0,130699.0,120539.0
4,Adair,IA,0.697,0.286,4183.0,2917.0,1197.0


In [138]:
df3 = df2.merge(votes, on=["county", "state"], how="inner")

In [139]:
df2.shape, df3.shape

((3219, 45), (3141, 50))

In [160]:
df3.head()

,FIPS,state,Series_Complete_18Plus,Series_Complete_18PlusPop_Pct,Series_Complete_65Plus,Series_Complete_65PlusPop_Pct,Series_Complete_Yes,Series_Complete_Pop_Pct,Census2019_12PlusPop,pct_college,...,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,percentage20_Donald_Trump,percentage20_Joe_Biden,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden
0,01001,AL,13216.0,30.8,4799.0,53.8,13470.0,24.1,47574.0,26.571573,...,73.6,20.9,5.5,0.0,7.6,0.715,0.270,27639.0,19764.0,7450.0
1,01003,AL,63308.0,36.0,29994.0,64.0,64146.0,28.7,192649.0,31.862459,...,81.5,12.3,5.8,0.4,7.5,0.762,0.223,108945.0,83055.0,24344.0
2,01005,AL,5647.0,28.8,2791.0,57.4,5673.0,23.0,21404.0,11.578713,...,71.8,20.8,7.3,0.1,17.6,0.536,0.456,10457.0,5605.0,4772.0
3,01007,AL,4192.0,23.5,1841.0,49.3,4232.0,18.9,19480.0,10.378526,...,76.8,16.1,6.7,0.4,8.3,0.784,0.207,9573.0,7508.0,1982.0
4,01009,AL,10333.0,23.2,4620.0,42.7,10438.0,18.1,49234.0,13.093413,...,82.0,13.5,4.2,0.4,7.7,0.896,0.096,27459.0,24595.0,2627.0


## Covid Transmission

In [165]:
df_covid = pd.read_csv("data/us-counties.csv", parse_dates=["date"])
df_covid = df_covid[df_covid["date"] == "2021-06-19"]
df_covid["FIPS"] = df_covid["geoid"].apply(lambda x: x[4:])

In [171]:
df4 = df3.merge(df_covid[
    ["FIPS", "cases_avg", "cases_avg_per_100k", "deaths_avg", "deaths_avg_per_100k"]
], on="FIPS")

In [172]:
df4.shape

(3132, 54)

In [173]:
df3.shape

(3141, 50)

### Output

In [175]:
df4.to_csv("combined_county_info.csv", index=False)